# Python Forecast Notebook

In [1]:
import pandas as pd

df = pd.read_csv("Supply_Chain_Dummy_1000_Rows.csv")

df.head()


,OrderID,OrderDate,PromisedDate,DeliveryDate,ProductID,Category,SupplierID,Warehouse,Customer,OrderedQty,DeliveredQty,UnitCost,SellingPrice,Revenue,COGS,InventoryValue,ForecastQty,OnTimeFlag,InFullFlag
0,ORD1,2023-12-04,2023-12-07,2023-12-10,P1010,Personal Care,S101,Sharjah DC,Lulu,347,338,41.32,69.63,23534.94,13966.16,16488.39,342,0,0
1,ORD2,2024-04-11,2024-04-14,2024-04-17,P1021,Electronics,S104,Sharjah DC,Noon,467,457,33.56,53.26,24339.82,15336.92,4523.40,438,0,0
2,ORD3,2024-01-21,2024-01-28,2024-01-31,P1046,Personal Care,S110,Abu Dhabi Hub,Noon,97,77,44.26,59.71,4597.67,3408.02,11381.60,101,0,0
3,ORD4,2023-08-08,2023-08-13,2023-08-12,P1049,Food,S103,Dubai Central,Spinneys,111,104,12.34,19.01,1977.04,1283.36,16725.28,133,1,0
4,ORD5,2024-02-12,2024-02-20,2024-02-22,P1014,Personal Care,S109,Abu Dhabi Hub,Carrefour,156,151,26.15,35.29,5328.79,3948.65,19355.82,182,0,0


In [3]:
df['OrderDate'] = pd.to_datetime(df['OrderDate'])
df['DeliveryDate'] = pd.to_datetime(df['DeliveryDate'])
df['PromisedDate'] = pd.to_datetime(df['PromisedDate'])

### Calculate OTIF in Python

In [5]:
df['OTIF'] = ((df['DeliveredQty'] >= df['OrderedQty']) &
              (df['DeliveryDate'] <= df['PromisedDate'])).astype(int)

otif_rate = df['OTIF'].mean()

print("OTIF Rate:", round(otif_rate * 100, 2), "%")


OTIF Rate: 1.7 %


### Demand Forecasting 

In [7]:
from sklearn.linear_model import LinearRegression
import numpy as np

# Monthly aggregation
monthly = df.resample('M', on='OrderDate')['OrderedQty'].sum().reset_index()

monthly['MonthIndex'] = np.arange(len(monthly))

monthly


C:\Users\Irine Susan Issac\AppData\Local\Temp\ipykernel_19772\1385692247.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly = df.resample('M', on='OrderDate')['OrderedQty'].sum().reset_index()


,OrderDate,OrderedQty,MonthIndex
0,2023-01-31,11519,0
1,2023-02-28,9108,1
2,2023-03-31,11524,2
3,2023-04-30,11906,3
4,2023-05-31,14207,4
5,2023-06-30,10443,5
6,2023-07-31,13311,6
7,2023-08-31,11670,7
8,2023-09-30,11198,8
9,2023-10-31,10268,9


#### Train regression model

In [9]:
X = monthly[['MonthIndex']]
y = monthly['OrderedQty']

model = LinearRegression()
model.fit(X, y)


LinearRegression()

#### Next 3 months Prediction

In [11]:
future = np.array([[len(monthly)],
                   [len(monthly)+1],
                   [len(monthly)+2]])

predictions = model.predict(future)

predictions


C:\Users\Irine Susan Issac\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([10999.89130435, 10968.8726087 , 10937.85391304])

##### Interpretation:

Demand is expected to stabilize around ~11,000 units per month.

##### The regression model predicts stable demand around 11K units per month, suggesting operational capacity planning should align with this baseline.

##### Demand Forecast Insight

The linear regression model predicts that monthly demand will stabilize
around 11,000 units. Despite low OTIF performance (1.7%), demand remains
consistent, indicating that operational inefficiency — not demand decline —
is the core business risk.